In [1]:
from tkinter import *

import threading
import time
import math
import grpc
# 导入自动生成的两个py文件
import router_pb2 as pb2
import router_pb2_grpc as pb2_grpc




conn_list = []
conn_dt = {}
mutex=threading.Lock()

routinginfo=[]
routinginfo.append((192,168,1,1,26,'A'))
routinginfo.append((128,0,1,1,24,'B'))
routinginfo.append((128,0,0,0,1,'C'))
routinginfo.append((0,0,0,0,32,'D'))

prefix=[]



class Router(pb2_grpc.RouterServicer):

    def hello_router(self, request, context):    
        from_ip = request.from_ip
        to_ip = request.to_ip  
        if from_ip not in conn_list:
            gui.listBox.insert(END, from_ip)
        print('connect from:',from_ip) 
        
        #发送
        if len(to_ip) !=0:
            recvlist=(to_ip.split('.', 3 ))
            gui.infoList.config(state=NORMAL)
            mutex.acquire()    
    ###x=int((i[0]*16777216+i[1]*65536+i[2]*256+i[3]*1)/pow(2,32-i[4]))
            y=int(recvlist[0])*16777216+int(recvlist[1])*65536+int(recvlist[2])*256+int(recvlist[3])*1
            y0=int(y/pow(2,32-routinginfo[0][4]))
            y1=int(y/pow(2,32-routinginfo[1][4]))
            y2=int(y/pow(2,32-routinginfo[2][4]))
            XX='D'
            if y0==prefix[0]:
                XX='A'
            if y1==prefix[1]:
                XX='B'
            if y2==prefix[2]:
                XX='C'       
            gui.infoList.insert(END, from_ip, 'name')
            gui.infoList.insert(END, '：\t')
            gui.infoList.insert(END, to_ip+' 端口'+XX, 'conment')
            gui.infoList.insert(END, '\n\n')
    ###  gui.infoList.insert(END, txt,'content')
    ###   gui.infoList.insert(END, from_id+'：\t'+to_ip+'\n\n', 'content')
            gui.infoList.config(state=DISABLED)
            mutex.release()        
        # 里面返回是response, 这个response内部只有一个字符串类型的result
        result = f"Server get message from  {from_ip}"
        # result需要放在response里面
        return pb2.response(result=result)


        
        
        
        
class GUI:
    def __init__(self, root):
        self.root = root
        self.leftFrame = Frame(self.root, width=20, height=60)
        self.leftFrame.grid(row=0, column=0)

        self.rightFrame = Frame(self.root, width=20, height=60)
        self.rightFrame.grid(row=0, column=1)

        Label(self.leftFrame, text='主机列表').grid(row=0, column=0)

        self.listBox = Listbox(self.leftFrame, width=15, height=10)
        self.listBox.grid(row=1, column=0)

        Label(self.leftFrame, text='路由表').grid(row=2, column=0)
        self.listBox2 = Listbox(self.leftFrame, width=15, height=10)
        self.listBox2.grid(row=3, column=0)
###
###        Label(self.leftFrame, text='ceshi').grid(row=2, column=0)

###        self.listBox2 = Listbox(self.leftFrame, width=15, height=10)
###        self.listBox2.grid(row=3, column=0)
###

###        self.entry = Entry(self.rightFrame, font=('Times New Roman', 18), width=30)                                                                                                                                                                                                                                                                                
###        self.entry.grid(row=0, column=0)

###        self.sendBtn = Button(self.rightFrame, text='发送', command=self.send, width=20)
###        self.sendBtn.grid(row=0, column=1)

        Label(self.rightFrame, text='路由器信息').grid(row=0, column=0)

        self.infoList = Text(self.rightFrame, width=40, height=30)
        self.infoList.grid(row=1, column=0)
        
        
        self.infoList.tag_config('name', foreground='red')
        self.infoList.tag_config('conment', foreground='grey')

###    def send(self):
###        _index = self.listBox.curselection()
###        conn_dt[self.listBox.get(_index)].sendall(self.entry.get().encode('utf-8'))
###        self.entry.delete(0, END)
def createGUI():
    global gui
    
    root = Tk()
    gui = GUI(root)
    root.title('模拟路由器原理-服务端')
    for i in routinginfo:
        x=int((i[0]*16777216+i[1]*65536+i[2]*256+i[3]*1)/pow(2,32-i[4]))
        prefix.append(x)
        gui.listBox2.insert(END,str(i[0])+'.'+str(i[1])+'.'+str(i[2])+'.'+str(i[3])+'/'+str(i[4])+':'+i[5])
    root.mainloop()
    
        
def grpcflow():
    from concurrent.futures import ThreadPoolExecutor
    grpc_server = grpc.server(ThreadPoolExecutor(max_workers=4))
    # 将服务注册到gRPC服务中
    pb2_grpc.add_RouterServicer_to_server(Router(), grpc_server)
    # 绑定ip和端口
    grpc_server.add_insecure_port("127.0.0.1:22533")
    # 启动服务
    grpc_server.start()
    try:
        while True:
            time.sleep(3600)
    except KeyboardInterrupt:
        # 当按下Ctrl+C, 终止服务
        grpc_server.stop(0)
if __name__ == '__main__':
    # t = threading.Thread(target=createGUI, args=(), name='GUI')
    # t.daemon = True
    # t.start()

    ###子网UI 
    t1 = threading.Thread(target=grpcflow, args=(), name='GRPC')
    t1.start()
    

    t2 = threading.Thread(target=createGUI, args=(), name='GUI')

    t2.start()


    
   

connect from: 127.0.0.58
connect from: 127.0.0.148
connect from: 127.0.0.108
connect from: 127.0.0.82
connect from: 127.0.0.234
connect from: 127.0.0.148
connect from: 127.0.0.108
connect from: 127.0.0.82
connect from: 127.0.0.234
connect from: 127.0.0.148
connect from: 127.0.0.108
connect from: 127.0.0.82
connect from: 127.0.0.234
connect from: 127.0.0.148
connect from: 127.0.0.108
connect from: 127.0.0.82
connect from: 127.0.0.234
connect from: 127.0.0.148
connect from: 127.0.0.108
connect from: 127.0.0.82
connect from: 127.0.0.234
connect from: 127.0.0.148
connect from: 127.0.0.108
connect from: 127.0.0.82
connect from: 127.0.0.234
connect from: 127.0.0.148
connect from: 127.0.0.108
connect from: 127.0.0.82
connect from: 127.0.0.234
connect from: 127.0.0.148
connect from: 127.0.0.108
connect from: 127.0.0.82
connect from: 127.0.0.234
connect from: 127.0.0.148
connect from: 127.0.0.108
connect from: 127.0.0.82
connect from: 127.0.0.234
connect from: 127.0.0.148
connect from: 127.0.0.